In [1]:
import os

import pandas as pd

In [2]:
%pwd

In [3]:
os.chdir("../")

In [4]:
%pwd

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path

In [6]:
from src.constants import *
from src.utils.common import read_yaml, create_directories


class ConfigurationManager:
    def __init__(self, 
                 config_filepath = CONFIG_FILE_PATH, 
                 schema_filepath = SCHEMA_FILE_PATH,
                params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.schema = read_yaml(schema_filepath)
        self.params = read_yaml(params_filepath)
        
        create_directories([self.config.artifacts_root])
        
    def get_data_transformation_config(self) -> DataTransformationConfig:
        
        config = self.config.data_transformation
        
        create_directories([config.root_dir])
        
        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path
        )
        return data_transformation_config
        

In [15]:
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from src.logging import logger


class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config
    
    def train_test_split(self):
        
        dataframe = pd.read_csv(self.config.data_path)
        train, test = train_test_split(dataframe, test_size=0.2)
        
        train.to_csv(os.path.join(self.config.root_dir, "train.csv"), index = False)
        test.to_csv(os.path.join(self.config.root_dir, "test.csv"), index = False)
        
        logger.info(f"Data is split into train and test")
        logger.info(f"Train shape: {train.shape}")
        logger.info(f"Test shape: {test.shape}")    
        

In [16]:
try:
    config = ConfigurationManager()
    data_transform_config = config.get_data_transformation_config()
    data_transform = DataTransformation(config=data_transform_config)
    data_transform.train_test_split()
except Exception as e:
    logger.exception(f"{e}")
    raise e